In [1]:
from __future__ import print_function, division

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)



    def build_generator(self):
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)


    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d global_step: %d[D loss: %f, acc.: %.2f%%] [G loss: %f]"
                      % (epoch, global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)



    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_bgan_fashion'):
            os.mkdir('images_bgan_fashion')
        fig.savefig("images_bgan_fashion/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=40, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 global_step: 1[D loss: 1.167716, acc.: 35.94%] [G loss: 0.666666]
epoch:0 global_step: 2[D loss: 0.631526, acc.: 64.06%] [G loss: 1.078489]
epoch:0 global_step: 3[D loss: 0.503884, acc.: 76.56%] [G loss: 1.370687]
epoch:0 global_step: 4[D loss: 0.485858, acc.: 73.44%] [G loss: 1.285066]
epoch:0 global_step: 5[D loss: 0.509791, acc.: 71.88%] [G loss: 1.020080]
epoch:0 global_step: 6[D loss: 0.414050, acc.: 78.91%] [G loss: 0.938021]
epoch:0 global_step: 7[D loss: 0.279461, acc.: 92.19%] [G loss: 1.147544]
epoch:0 global_step: 8[D loss: 0.239635, acc.: 91.41%] [G loss: 0.863792]
epoch:0 global_step: 9[D loss: 0.210774, acc.: 93.75%] [G loss: 0.818848]
epoch:0 global_step: 10[D loss: 0.269763, acc.: 88.28%] [G loss: 0.798845]
epoch:0 global_step: 11[D loss: 0.316336, acc.: 88.28%] [G loss: 0.982788]
epoch:0 global_step: 12[D loss: 0.410345, acc.: 77.34%] [G loss: 1.610200]
epoch:0 global_step: 13[D loss: 0.665258, acc.: 67.19%] [G loss: 2.995780]
epoch:0 global_step: 14[D loss: 0.

epoch:0 global_step: 111[D loss: 0.884690, acc.: 48.44%] [G loss: 1.355947]
epoch:0 global_step: 112[D loss: 0.472377, acc.: 77.34%] [G loss: 1.351676]
epoch:0 global_step: 113[D loss: 0.665152, acc.: 57.03%] [G loss: 1.157925]
epoch:0 global_step: 114[D loss: 0.640542, acc.: 66.41%] [G loss: 1.836904]
epoch:0 global_step: 115[D loss: 0.751864, acc.: 57.03%] [G loss: 1.058203]
epoch:0 global_step: 116[D loss: 0.665447, acc.: 65.62%] [G loss: 2.010484]
epoch:0 global_step: 117[D loss: 0.648038, acc.: 64.84%] [G loss: 1.822374]
epoch:0 global_step: 118[D loss: 0.739807, acc.: 57.03%] [G loss: 1.156386]
epoch:0 global_step: 119[D loss: 0.780454, acc.: 57.81%] [G loss: 1.640127]
epoch:0 global_step: 120[D loss: 0.696596, acc.: 62.50%] [G loss: 1.469818]
epoch:0 global_step: 121[D loss: 0.755294, acc.: 58.59%] [G loss: 1.021950]
epoch:0 global_step: 122[D loss: 0.595556, acc.: 66.41%] [G loss: 1.183676]
epoch:0 global_step: 123[D loss: 0.550455, acc.: 67.19%] [G loss: 2.157161]
epoch:0 glob

...

epoch:39 global_step: 37016[D loss: 0.418012, acc.: 79.69%] [G loss: 1.394516]
epoch:39 global_step: 37017[D loss: 0.430807, acc.: 80.47%] [G loss: 2.464068]
epoch:39 global_step: 37018[D loss: 0.356419, acc.: 83.59%] [G loss: 6.139239]
epoch:39 global_step: 37019[D loss: 0.298816, acc.: 84.38%] [G loss: 1.577645]
epoch:39 global_step: 37020[D loss: 0.345258, acc.: 84.38%] [G loss: 2.912471]
epoch:39 global_step: 37021[D loss: 0.284117, acc.: 89.84%] [G loss: 3.104908]
epoch:39 global_step: 37022[D loss: 0.323022, acc.: 86.72%] [G loss: 2.608201]
epoch:39 global_step: 37023[D loss: 0.315601, acc.: 87.50%] [G loss: 3.136493]
epoch:39 global_step: 37024[D loss: 0.715068, acc.: 65.62%] [G loss: 1.732592]
epoch:39 global_step: 37025[D loss: 0.516013, acc.: 78.91%] [G loss: 1.654987]
epoch:39 global_step: 37026[D loss: 0.551324, acc.: 68.75%] [G loss: 1.577344]
epoch:39 global_step: 37027[D loss: 0.368311, acc.: 86.72%] [G loss: 3.198025]
epoch:39 global_step: 37028[D loss: 0.404563, acc.: 

epoch:39 global_step: 37121[D loss: 0.883452, acc.: 51.56%] [G loss: 0.810695]
epoch:39 global_step: 37122[D loss: 0.628183, acc.: 66.41%] [G loss: 5.391873]
epoch:39 global_step: 37123[D loss: 1.092883, acc.: 37.50%] [G loss: 2.495187]
epoch:39 global_step: 37124[D loss: 0.528366, acc.: 68.75%] [G loss: 4.277795]
epoch:39 global_step: 37125[D loss: 0.632973, acc.: 62.50%] [G loss: 1.210298]
epoch:39 global_step: 37126[D loss: 0.365753, acc.: 85.16%] [G loss: 2.872327]
epoch:39 global_step: 37127[D loss: 0.444317, acc.: 77.34%] [G loss: 2.365132]
epoch:39 global_step: 37128[D loss: 0.637990, acc.: 67.97%] [G loss: 1.925909]
epoch:39 global_step: 37129[D loss: 0.417570, acc.: 78.91%] [G loss: 5.493629]
epoch:39 global_step: 37130[D loss: 0.321161, acc.: 84.38%] [G loss: 5.370971]
epoch:39 global_step: 37131[D loss: 0.437410, acc.: 82.03%] [G loss: 2.138187]
epoch:39 global_step: 37132[D loss: 0.400269, acc.: 79.69%] [G loss: 1.864119]
epoch:39 global_step: 37133[D loss: 0.320443, acc.: 

epoch:39 global_step: 37227[D loss: 0.544015, acc.: 73.44%] [G loss: 0.784866]
epoch:39 global_step: 37228[D loss: 0.502288, acc.: 74.22%] [G loss: 3.609055]
epoch:39 global_step: 37229[D loss: 0.286058, acc.: 90.62%] [G loss: 7.263787]
epoch:39 global_step: 37230[D loss: 0.424227, acc.: 82.03%] [G loss: 1.617352]
epoch:39 global_step: 37231[D loss: 0.224655, acc.: 92.97%] [G loss: 4.678162]
epoch:39 global_step: 37232[D loss: 0.345603, acc.: 88.28%] [G loss: 2.356066]
epoch:39 global_step: 37233[D loss: 0.432138, acc.: 82.03%] [G loss: 4.632406]
epoch:39 global_step: 37234[D loss: 0.332224, acc.: 87.50%] [G loss: 5.627649]
epoch:39 global_step: 37235[D loss: 0.368615, acc.: 82.81%] [G loss: 2.220840]
epoch:39 global_step: 37236[D loss: 0.748849, acc.: 50.78%] [G loss: 2.451978]
epoch:39 global_step: 37237[D loss: 0.457857, acc.: 78.12%] [G loss: 3.442947]
epoch:39 global_step: 37238[D loss: 0.348844, acc.: 87.50%] [G loss: 2.546106]
epoch:39 global_step: 37239[D loss: 0.667468, acc.: 

epoch:39 global_step: 37334[D loss: 0.482554, acc.: 77.34%] [G loss: 1.656579]
epoch:39 global_step: 37335[D loss: 0.609956, acc.: 67.97%] [G loss: 2.818439]
epoch:39 global_step: 37336[D loss: 0.484045, acc.: 80.47%] [G loss: 2.276737]
epoch:39 global_step: 37337[D loss: 0.158440, acc.: 96.88%] [G loss: 4.170207]
epoch:39 global_step: 37338[D loss: 0.197580, acc.: 95.31%] [G loss: 2.537848]
epoch:39 global_step: 37339[D loss: 0.270083, acc.: 92.97%] [G loss: 4.503207]
epoch:39 global_step: 37340[D loss: 0.229401, acc.: 91.41%] [G loss: 3.346028]
epoch:39 global_step: 37341[D loss: 0.801149, acc.: 57.81%] [G loss: 1.241363]
epoch:39 global_step: 37342[D loss: 0.595724, acc.: 71.09%] [G loss: 3.992813]
epoch:39 global_step: 37343[D loss: 0.406050, acc.: 85.16%] [G loss: 1.573221]
epoch:39 global_step: 37344[D loss: 0.282658, acc.: 93.75%] [G loss: 1.014428]
epoch:39 global_step: 37345[D loss: 0.298964, acc.: 86.72%] [G loss: 1.545386]
epoch:39 global_step: 37346[D loss: 0.724930, acc.: 

epoch:39 global_step: 37442[D loss: 0.367598, acc.: 84.38%] [G loss: 1.559969]
epoch:39 global_step: 37443[D loss: 0.268309, acc.: 89.84%] [G loss: 2.094231]
epoch:39 global_step: 37444[D loss: 0.530439, acc.: 72.66%] [G loss: 1.150764]
epoch:39 global_step: 37445[D loss: 0.319388, acc.: 92.97%] [G loss: 2.902509]
epoch:39 global_step: 37446[D loss: 0.326786, acc.: 85.94%] [G loss: 2.715909]
epoch:39 global_step: 37447[D loss: 0.337529, acc.: 90.62%] [G loss: 3.280378]
epoch:39 global_step: 37448[D loss: 0.512162, acc.: 78.91%] [G loss: 1.872378]
epoch:39 global_step: 37449[D loss: 0.369243, acc.: 89.06%] [G loss: 2.995973]
epoch:39 global_step: 37450[D loss: 0.528536, acc.: 74.22%] [G loss: 2.780003]
epoch:39 global_step: 37451[D loss: 0.322471, acc.: 89.84%] [G loss: 2.667012]
epoch:39 global_step: 37452[D loss: 0.364404, acc.: 89.84%] [G loss: 3.886928]
epoch:39 global_step: 37453[D loss: 0.411292, acc.: 82.81%] [G loss: 2.292678]
epoch:39 global_step: 37454[D loss: 0.588710, acc.: 